In [1]:
import numpy as np
import pandas as pd
import math
import time
from scipy.stats import laplace
from multiprocessing.pool import ThreadPool

# Pré-processamento do dataset

In [2]:
def preProcessamento(dataset):
    return np.array(dataset)

# Funções de consultas

In [3]:
def Q1(dataset):    
    return np.mean(dataset[:,0])

In [4]:
def Q2(dataset):
    return np.sum(dataset[:,10])

In [5]:
def Q3(dataset):
    return np.mean(dataset[:,12])

In [6]:
def Q4(dataset):
    contIncome50k = 0
    for registro in dataset:
        if(registro[14] == '>50K'):
            contIncome50k = contIncome50k + 1
    return contIncome50k

# Obtenção da sensibilidade global de todas as funções de consulta

In [7]:
def sensibilidadeGlobal(funcaoConsulta, dataset):
    sensibilidade = 0
    respostaConsulta = funcaoConsulta(dataset)
    for registro in range(0, dataset.shape[0]):
        datasetVizinho = np.copy(dataset)
        datasetVizinho = np.delete(datasetVizinho, registro, 0)
        respostaVizinho = funcaoConsulta(datasetVizinho)
        sens = abs(respostaConsulta - respostaVizinho)
        if(sens >= sensibilidade):
            sensibilidade = sens
        time.sleep(0.00001)
        if(registro%100 == 0):
            print(registro)
    return sensibilidade

In [8]:
def sensibilidadeGlobalMult(funcaoConsulta, dataset):
    pool = ThreadPool(processes = 100)
    async_call = pool.apply_async(sensibilidadeGlobal, (funcaoConsulta, dataset, ))
    return async_call.get()

In [9]:
def adquirirSensibilidades(consultas, dataset):
    sensibilidades = []
    for consulta in consultas:
        sensibilidades.append(sensibilidadeGlobalMult(consulta, dataset))
    return sensibilidades

# Geração da resposta com ruído

In [10]:
def respostaRuido(budget, sensibilidade, funcaoConsulta, dataset):
    respostaConsulta = funcaoConsulta(dataset)
    probOriginal = laplace.pdf(0, loc = 0, scale = sensibilidade/budget)
    privacidade = False
    respostaRuido = 0
    while(not privacidade):
        ruido = laplace.rvs(loc = 0, scale = sensibilidade/budget)
        resposta = respostaConsulta + ruido
        probRuido = laplace.pdf(ruido, loc = 0, scale = sensibilidade/budget)
        if(probOriginal <= (math.e**budget)*probRuido):
            privacidade = True
            respostaRuido = resposta
    return respostaRuido

# Gerar CSV

In [11]:
def posProcessamento(budgets, respostas, sensibilidades):
    datasetFinal = []
    for i in range(0, len(budgets)):
        registro = [budgets[i]]
        for j in range(0, len(respostas[i])):
            registro.append(respostas[i][j])
        for j in range(0, len(sensibilidades)):
            registro.append(sensibilidades[j])
        datasetFinal.append(registro)
    datasetFinal = pd.DataFrame(datasetFinal, columns=['budget', 'result_q1', 'result_q2', 'result_q3', 'result_q4', 
                                                       'sens _q1', 'sens_q2', 'sens_q3', 'sens_q4'])
    arquivo = 'result.csv'
    datasetFinal.to_csv(arquivo, index=False)

# Execução

In [12]:
dataset = pd.read_csv('adult.csv')
dataset = preProcessamento(dataset)
dataset = dataset[0:10000, :]

In [13]:
consultas = [Q1, Q2, Q3, Q4]
budgets = [0.1, 1, 10]
sensibilidades = adquirirSensibilidades(consultas, dataset)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
0
100
200
300
400
500
60

In [14]:
respostas = []
for budget in budgets:
    respostaBudget = []
    for i in range(0, len(consultas)):
        resposta = respostaRuido(budget/len(consultas), sensibilidades[i], consultas[i], dataset)
        respostaBudget.append(resposta)
    respostas.append(respostaBudget)
posProcessamento(budgets, respostas, sensibilidades)

In [15]:
for i in range(0, len(budgets)):
    for j in range(0, len(consultas)):
        print('Budget:', budgets[i])
        print('Consulta: Q'+str(j+1))
        print('Sensibilidade:', sensibilidades[j])
        print('Resposta Original:', consultas[j](dataset))
        print('Resposta + Ruído:', respostas[i][j],'\n')

Budget: 0.1
Consulta: Q1
Sensibilidade: 0.00022482248224831025
Resposta Original: 1.752
Resposta + Ruído: 1.7518063693684789 

Budget: 0.1
Consulta: Q2
Sensibilidade: 4
Resposta Original: 2018
Resposta + Ruído: 2016.8041217009759 

Budget: 0.1
Consulta: Q3
Sensibilidade: 0.0002040304030401785
Resposta Original: 1.9599
Resposta + Ruído: 1.9597220144408145 

Budget: 0.1
Consulta: Q4
Sensibilidade: 1
Resposta Original: 2379
Resposta + Ruído: 2378.793227426411 

Budget: 1
Consulta: Q1
Sensibilidade: 0.00022482248224831025
Resposta Original: 1.752
Resposta + Ruído: 1.751821776645695 

Budget: 1
Consulta: Q2
Sensibilidade: 4
Resposta Original: 2018
Resposta + Ruído: 2019.0717177142353 

Budget: 1
Consulta: Q3
Sensibilidade: 0.0002040304030401785
Resposta Original: 1.9599
Resposta + Ruído: 1.959901122054788 

Budget: 1
Consulta: Q4
Sensibilidade: 1
Resposta Original: 2379
Resposta + Ruído: 2378.373824842768 

Budget: 10
Consulta: Q1
Sensibilidade: 0.00022482248224831025
Resposta Original: 1.7